# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [10]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score

In [4]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
spaceship = spaceship.drop(columns=['PassengerId', 'Name', 'Cabin'])
spaceship['CryoSleep'] = spaceship['CryoSleep'].fillna(spaceship['CryoSleep'].mode()[0])
spaceship['HomePlanet'] = spaceship['HomePlanet'].fillna(spaceship['HomePlanet'].mode()[0])
spaceship['Destination'] = spaceship['Destination'].fillna(spaceship['Destination'].mode()[0])
spaceship['VIP'] = spaceship['VIP'].fillna(spaceship['VIP'].mode()[0])

In [6]:
numerical_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
spaceship[numerical_cols] = spaceship[numerical_cols].fillna(spaceship[numerical_cols].median())

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [11]:
label_encoder = LabelEncoder()
categorical_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']
for col in categorical_cols:
    spaceship[col] = label_encoder.fit_transform(spaceship[col])
scaler = StandardScaler()
spaceship[numerical_cols] = scaler.fit_transform(spaceship[numerical_cols])


In [12]:
X = spaceship.drop(columns=['Transported'])
y = spaceship['Transported']

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

- Evaluate your model

In [15]:
baseline_model = BaggingClassifier(random_state=0)
baseline_model.fit(X_train, y_train)

# Predict on the test set
y_pred_baseline = baseline_model.predict(X_test)

# Evaluate the model
baseline_accuracy = accuracy_score(y_test, y_pred_baseline)
print(f"Baseline Test Accuracy: {baseline_accuracy}")

Baseline Test Accuracy: 0.7726226993865031


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [17]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_samples': [0.5, 0.75, 1.0],
    'bootstrap': [True, False],
    'bootstrap_features': [True, False]
}


- Run Grid Search

In [18]:
grid_search = GridSearchCV(estimator=BaggingClassifier(random_state=0), 
                           param_grid=param_grid, 
                           cv=5, 
                           scoring='accuracy', 
                           n_jobs=-1, 
                           verbose=2)

In [19]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=5, estimator=BaggingClassifier(random_state=0), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'bootstrap_features': [True, False],
                         'max_samples': [0.5, 0.75, 1.0],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy', verbose=2)

- Evaluate your model

In [20]:
best_model = grid_search.best_estimator_

# Predict on the test set using the best model
y_pred = best_model.predict(X_test)

# Evaluate the model
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy of the best model: {test_accuracy}")


Test Accuracy of the best model: 0.7848926380368099
